# Chapter16 - 코루틴

## 16.2 코루틴으로 사용되는 제너레이터의 기본 동작

In [2]:
def simple_coroutine():
    print('-> coroutine started')
    x = yield
    print('-> coroutine recevied:', x)
    
my_coro = simple_coroutine()
my_coro

<generator object simple_coroutine at 0x110f9a5e8>

In [3]:
next(my_coro)

-> coroutine started


In [4]:
my_coro.send(42)

-> coroutine recevied: 42


StopIteration: 

In [ ]:
my_coro = simple_coroutine()
my_coro.send(1729)

In [ ]:
def simple_coro2(a):
    print('-> Started: a =', a)
    b = yield a
    print('-> Received: b =', b)
    c = yield a + b
    print('-> Received: c =', c)
    
my_coro2 = simple_coro2(14)

from inspect import getgeneratorstate
getgeneratorstate(my_coro2)

In [ ]:
next(my_coro2)

In [ ]:
getgeneratorstate(my_coro2)

In [ ]:
my_coro2.send(28)

In [ ]:
my_coro2.send(99)

In [ ]:
getgeneratorstate(my_coro2)

## 16.3 예제: 이동 평균을 계산하는 코루틴

In [ ]:
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total / count
        
coro_avg = averager()
next(coro_avg)
coro_avg.send(10)

In [ ]:
coro_avg.send(30)

In [ ]:
coro_avg.send(5)

## 16.4 코루틴을 기동하기 위한 데코레이터

In [ ]:
from functools import wraps

def coroutine(func):
    @wraps(func)
    def primer(*ars, **kwargs):
        gen = func(*ars, **kwargs)
        next(gen)
        return gen
    
    return primer

@coroutine
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total / count
        
coro_avg = averager()
coro_avg.send(10)

In [ ]:
coro_avg.send(30)

## 16.5 코루틴 종료와 예외 처리

In [ ]:
coro_avg = averager()
coro_avg.send(40)

In [ ]:
coro_avg.send(50)

In [ ]:
coro_avg.send('spam')

In [ ]:
coro_avg.send(60)

In [ ]:
class DemoException(Exception):
    """ """

def demo_exc_handling():
    print('-> coroutine started')
    while True:
        try: 
            x = yield
        except DemoException:
            print('*** DemoException handled. Coroutining...')
        else:
            print('-> coroutine received: {!r}'.format(x))
            
    raise RuntimeError('This line shoud never run.')
    
exc_coro = demo_exc_handling()
next(exc_coro)

In [ ]:
exc_coro.send(11)

In [ ]:
exc_coro.send(22)

In [ ]:
exc_coro.close()

In [ ]:
getgeneratorstate(exc_coro)

In [ ]:
exc_coro = demo_exc_handling()
next(exc_coro)
exc_coro.send(11)

In [ ]:
exc_coro.throw(DemoException)

In [ ]:
getgeneratorstate(exc_coro)

In [ ]:
exc_coro.send(22)

In [ ]:
exc_coro = demo_exc_handling()
next(exc_coro)
exc_coro.send(11)

In [ ]:
exc_coro.throw(ZeroDivisionError)

In [ ]:
getgeneratorstate(exc_coro)

In [ ]:
def demo_finally():
    print('-> coroutine started')
    try:
        while True:
            try:
                x = yield
            except DemoException:
                print('*** DemoException handled. Coroutining...')
            else:
                print('-> coroutine received: {!r}'.format(x))
    finally:
        print('-> coroutine ending')

In [ ]:
exc_coro = demo_finally()
next(exc_coro)
exc_coro.send(11)
exc_coro.throw(ZeroDivisionError)

## 16.6 코루틴에서 값 반환하기

In [ ]:
from collections import namedtuple

Result = namedtuple('Result', 'count average')

def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield
        if term is None:
            break
        total += term
        count += 1
        average = total / count
    return Result(count, average)

coro_avg = averager()
next(coro_avg)
coro_avg.send(10)
coro_avg.send(30)
coro_avg.send(6.5)
coro_avg.send(None)

In [ ]:
coro_avg = averager()
next(coro_avg)
coro_avg.send(10)
coro_avg.send(30)
coro_avg.send(6.5)
try:
    coro_avg.send(None)
except StopIteration as exc:
    result = exc.value
    
result

## 16.7 yield from 사용하기

In [ ]:
def gen():
    for c in 'AB':
        yield c
    for i in range(1, 3):
        yield i
        
list(gen())

In [ ]:
def gen():
    yield from 'AB'
    yield from range(1, 3)
    
list(gen())

In [ ]:
def chain(*iterables):
    for it in iterables:
        yield from it
        
s = 'ABC'
t = tuple(range(3))
list(chain(s, t))

In [ ]:
from collections import namedtuple

Result = namedtuple('Result', 'count average')

def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield
        if term is None:
            break
        total += term
        count += 1
        average = total / count
    return Result(count, average)

def grouper(results, key):
    while True:
        results[key] = yield from averager()
        
def main(data):
    results = {}
    for key, values in data.items():
        group = grouper(results, key)
        next(group)
        for value in values:
            group.send(value)
        group.send(None)
        
    print(results)
    report(results)
    
def report(results):
    for key, result in sorted(results.items()):
        group, uint = key.split(';')
        print('{:2} {:5} averaging {:.2f}{}'.format(result.count, group, result.average, uint))
        
data = {
    'girls;kg':
        [40.9, 38.5, 44.3, 42.2, 45.2, 41.7, 44.5, 38.0, 40.6, 44.5],
    'girls;m':
        [1.6, 1.51, 1.4, 1.3, 1.41, 1.39, 1.33, 1.46, 1.45, 1.43],
    'boys;kg':
        [39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3],
    'boys;m':
        [1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46],
}

main(data)

## 16.9 사용 사례: 이산 이벤트 시뮬레이션을 위한 코루틴

### 16.9.2 택시 집단 시뮬레이션

In [5]:
from collections import namedtuple

Event = namedtuple('Event', 'time proc action')

def taxi_process(ident, trips, start_time=0):
    time = yield Event(start_time, ident, 'leave garage')
    for i in range(trips):
        time = yield Event(time, ident, 'pick up passenger')
        time = yield Event(time, ident, 'drop off passenger')
    yield Event(time, ident, 'going home')
    
taxi = taxi_process(ident=13, trips=2, start_time=0)
event = next(taxi)
event

Event(time=0, proc=13, action='leave garage')

In [6]:
event = taxi.send(event.time + 7)
event

Event(time=7, proc=13, action='pick up passenger')

In [7]:
event = taxi.send(event.time + 23)
event

Event(time=30, proc=13, action='drop off passenger')

In [8]:
event = taxi.send(event.time + 23)
event

Event(time=53, proc=13, action='pick up passenger')

In [9]:
event = taxi.send(event.time + 23)
event

Event(time=76, proc=13, action='drop off passenger')

In [10]:
event = taxi.send(event.time + 23)
event

Event(time=99, proc=13, action='going home')

In [11]:
event = taxi.send(event.time + 23)
event

StopIteration: 

In [22]:
import queue
import random

SEARCH_DURATION = 5
TRIP_DURATION = 20

def compute_duration(previous_action):
    """Compute action duration using exponential distribution"""
    if previous_action in ['leave garage', 'drop off passenger']:
        # new state is prowling
        interval = SEARCH_DURATION
    elif previous_action == 'pick up passenger':
        # new state is trip
        interval = TRIP_DURATION
    elif previous_action == 'going home':
        interval = 1
    else:
        raise ValueError('Unknown previous_action: %s' % previous_action)
    return int(random.expovariate(1/interval)) + 1

class Simulator:
    def __init__(self, procs_map):
        self.events = queue.PriorityQueue()
        self.procs = dict(procs_map)
        
    def run(self, end_time):
        for _, proc in sorted(self.procs.items()):
            first_event = next(proc)
            self.events.put(first_event)
            
        sim_time = 0
        while sim_time < end_time:
            if self.events.empty():
                print('*** end of envents ***')
                break
            
            current_event = self.events.get()
            sim_time, proc_id, previous_action = current_event
            print('taxi:', proc_id, proc_id * ' ', current_event)
            active_proc = self.procs[proc_id]
            next_time = sim_time + compute_duration(previous_action)
            try:
                next_event = active_proc.send(next_time)
            except StopIteration:
                del self.procs[proc_id]
            else:
                self.events.put(next_event)
        else:
            msg = '*** end of simulation time: {} event pending ***'
            print(msg.format(self.events.qsize()))
            
taxis = {0: taxi_process(ident=0, trips=2, start_time=0),
         1: taxi_process(ident=1, trips=4, start_time=5),
         2: taxi_process(ident=2, trips=6, start_time=10)}

sim = Simulator(taxis)
sim.run(200)

taxi: 0  Event(time=0, proc=0, action='leave garage')
taxi: 1   Event(time=5, proc=1, action='leave garage')
taxi: 0  Event(time=6, proc=0, action='pick up passenger')
taxi: 1   Event(time=7, proc=1, action='pick up passenger')
taxi: 2    Event(time=10, proc=2, action='leave garage')
taxi: 2    Event(time=22, proc=2, action='pick up passenger')
taxi: 1   Event(time=30, proc=1, action='drop off passenger')
taxi: 1   Event(time=34, proc=1, action='pick up passenger')
taxi: 1   Event(time=38, proc=1, action='drop off passenger')
taxi: 2    Event(time=42, proc=2, action='drop off passenger')
taxi: 0  Event(time=44, proc=0, action='drop off passenger')
taxi: 0  Event(time=45, proc=0, action='pick up passenger')
taxi: 1   Event(time=45, proc=1, action='pick up passenger')
taxi: 1   Event(time=50, proc=1, action='drop off passenger')
taxi: 1   Event(time=52, proc=1, action='pick up passenger')
taxi: 2    Event(time=62, proc=2, action='pick up passenger')
taxi: 1   Event(time=72, proc=1, actio